In [1]:
import pandas as pd

In [2]:
SELECTED_CONCEPTS = [1, 4, 6, 7, 10, 14, 15, 20, 21, 23, 25, 29, 30, 35, 36, 38, 40, 44, 45,
                     50, 51, 53, 54, 56, 57, 59, 63, 64, 69, 70, 72, 75, 80, 84, 90, 91, 93,
                     99, 101, 106, 110, 111, 116, 117, 119, 125, 126, 131, 132, 134, 145, 149,
                     151, 152, 153, 157, 158, 163, 164, 168, 172, 178, 179, 181, 183, 187, 188,
                     193, 194, 196, 198, 202, 203, 208, 209, 211, 212, 213, 218, 220, 221, 225,
                     235, 236, 238, 239, 240, 242, 243, 244, 249, 253, 254, 259, 260, 262, 268,
                     274, 277, 283, 289, 292, 293, 294, 298, 299, 304, 305, 308, 309, 310, 311]

# Without shape/size
SELECTED_CONCEPTS_V2 = [1, 4, 6, 7, 10, 14, 15, 20, 21, 23, 25, 29, 30, 35, 36, 38, 40, 44, 45,
                        50, 51, 53, 54, 56, 57, 59, 63, 64, 69, 70, 72, 75, 80, 84, 90, 91, 93,
                        99, 101, 106, 110, 111, 116, 117, 119, 125, 126, 131, 132, 134, 145, 149,
                        151, 152, 153, 157, 158, 163, 164, 168, 172, 178, 179, 181, 183, 187, 188,
                        193, 194, 196, 198, 202, 203, 208, 209, 211, 212, 213, 236, 238, 239, 240,
                        242, 243, 244, 249, 253, 254, 259, 260, 262, 268, 274, 277, 283, 289, 292,
                        293, 294, 298, 299, 304, 305, 308, 309, 310, 311]

In [3]:
attribute_df = pd.read_csv('datasets/CUB/attributes.txt', sep=' ', header=None, names=['attribute_id', 'attribute_name']).drop(columns=['attribute_id'])
attribute_df = attribute_df.iloc[SELECTED_CONCEPTS_V2]
fine_grained_parts = ['bill', 'upperpart', 'underpart', 'head', 'back', 'beak', 'belly', 'breast',
                      'crown', 'eye', 'forehead', 'leg', 'nape', 'tail', 'throat', 'wing', 'primary']
coarse_grained_parts = ['head', 'back', 'belly', 'breast', 'leg', 'tail', 'wing']
fine2coarse =  {
    'bill': 'head',
    'upperpart': 'breast',
    'underpart': 'belly',
    'head': 'head',
    'back': 'back',
    'beak': 'head',
    'belly': 'belly',
    'breast': 'breast',
    'crown': 'head',
    'eye': 'head',
    'forehead': 'head',
    'leg': 'leg',
    'nape': 'back',
    'tail': 'tail',
    'throat': 'breast',
    'wing': 'wing',
    'primary': 'breast'
}

In [4]:
part_pattern = '|'.join(fine_grained_parts)
attribute_df['part_name'] = attribute_df['attribute_name'].str.extract(f'({part_pattern})')
attribute_df['part_name'] = attribute_df['part_name'].replace(fine2coarse)
attribute_df['part_id'] = attribute_df['part_name'].map(coarse_grained_parts.index)
attribute_df

,attribute_name,part_name,part_id
1,has_bill_shape::dagger,head,0
4,has_bill_shape::hooked_seabird,head,0
6,has_bill_shape::all-purpose,head,0
7,has_bill_shape::cone,head,0
10,has_wing_color::brown,wing,6
...,...,...,...
305,has_crown_color::white,head,0
308,has_wing_pattern::solid,wing,6
309,has_wing_pattern::spotted,wing,6
310,has_wing_pattern::striped,wing,6


In [ ]:
attribute_df['part_id'].to_numpy()

In [ ]:
import torch

In [ ]:
torch.dot